In [1]:
!pip install pulp 

In [4]:
import pandas as pd
import pulp

employees_df = pd.read_excel('/home/wissal/Téléchargements/Updated_Employees_Data.xlsx')  
tasks_df = pd.read_excel('/home/wissal/Téléchargements/New_Tasks_Data.xlsx')  

prob = pulp.LpProblem("TaskAssignment", pulp.LpMaximize)

assignment_vars = pulp.LpVariable.dicts("assignment",
                                        ((task_id, emp_id) for task_id in tasks_df.index for emp_id in employees_df.index),
                                        cat='Binary')

prob += pulp.lpSum([assignment_vars[(task_id, emp_id)] for task_id in tasks_df.index for emp_id in employees_df.index])

for task_id in tasks_df.index:
    task = tasks_df.loc[task_id]
    prob += pulp.lpSum([assignment_vars[(task_id, emp_id)] for emp_id in employees_df.index]) == 1

for emp_id in employees_df.index:
    prob += pulp.lpSum([assignment_vars[(task_id, emp_id)] for task_id in tasks_df.index]) <= 1

for task_id in tasks_df.index:
    for emp_id in employees_df.index:
        employee = employees_df.loc[emp_id]
        task = tasks_df.loc[task_id]
        has_skill = task['Required_Skill'] in employee['Skills'].split(', ')
        is_available = task['Assembly_Line'] in map(int, employee['Available_Days'].split(', '))
        if not (has_skill and is_available):
            prob += assignment_vars[(task_id, emp_id)] == 0

prob.solve()

assignments = []
for task_id in tasks_df.index:
    for emp_id in employees_df.index:
        if pulp.value(assignment_vars[(task_id, emp_id)]) == 1:
            assignments.append({"Task_ID": task_id, 
                                "Employee_ID": emp_id})

assignment_df = pd.DataFrame(assignments)

assignment_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wissal/anaconda3/envs/myenv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2e704c14ae8c4214b01ccb264fc7b3b3-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2e704c14ae8c4214b01ccb264fc7b3b3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1315 COLUMNS
At line 10940 RHS
At line 12251 BOUNDS
At line 13932 ENDATA
Problem MODEL has 1310 rows, 1680 columns and 4584 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01



,Task_ID,Employee_ID
0,0,11
1,1,37
2,2,29
3,3,19
4,4,23
5,5,27
6,6,16
7,7,0
8,8,43
9,10,52
